In [1]:
import cv2
import csv
import mediapipe as mp
import cvzone
import time

In [2]:
# Initialize mediapipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.8)
mp_drawing = mp.solutions.drawing_utils

class MCQ:
    def __init__(self, data):
        self.question = data[0]
        self.choices = data[1:5]
        self.answer = int(float(data[5]))
        self.userAns = None

    def update(self, cursor, bboxs):
        for i, bbox in enumerate(bboxs):
            x1, y1, x2, y2 = bbox
            if x1 < cursor[0] < x2 and y1 < cursor[1] < y2:
                self.userAns = i + 1
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0))

# Read the CSV file and create MCQ objects
mcqList = []
with open('Mcqs.csv', newline='\n') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        mcqList.append(MCQ(row))

cap = cv2.VideoCapture(0)

qNo = 0
qTotal = len(mcqList)

while True:
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get the tip of the index and middle finger
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            
            # Convert the coordinates to pixel values
            h, w, c = img.shape
            index_tip = (int(index_finger_tip.x * w), int(index_finger_tip.y * h))
            middle_tip = (int(middle_finger_tip.x * w), int(middle_finger_tip.y * h))
            
            cursor = ((index_tip[0] + middle_tip[0]) // 2, (index_tip[1] + middle_tip[1]) // 2)
            length = ((index_tip[0] - middle_tip[0]) ** 2 + (index_tip[1] - middle_tip[1]) ** 2) ** 0.5

            # Draw the fingertips and the line connecting them
            cv2.circle(img, index_tip, 10, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, middle_tip, 10, (255, 0, 255), cv2.FILLED)
            cv2.line(img, index_tip, middle_tip, (255, 0, 255), 3)
            cv2.circle(img, cursor, 15, (0, 255, 0), cv2.FILLED)

            # Check for selection
            if length < 50:  # Adjust the threshold to your convenience
                if qNo < qTotal:
                    mcq = mcqList[qNo]
                    bboxs = [(25, 100, 140, 180), (270, 100, 385, 180), (25, 250, 140, 330), (270, 250, 385, 330)]  # Adjusted bounding box coordinates
                    mcq.update(cursor, bboxs)
                    if mcq.userAns is not None:
                        time.sleep(1)
                        qNo += 1

    # Draw the question and choices on the frame
    if qNo < qTotal:
        mcq = mcqList[qNo]
        img, bbox = cvzone.putTextRect(img, mcq.question, [25, 50], 2, 2, offset=30, border=5)
        img, bbox1 = cvzone.putTextRect(img, mcq.choices[0], [50, 150], 2, 2, offset=30, border=5)
        img, bbox2 = cvzone.putTextRect(img, mcq.choices[1], [300, 150], 2, 2, offset=30, border=5)
        img, bbox3 = cvzone.putTextRect(img, mcq.choices[2], [50, 300], 2, 2, offset=30, border=5)
        img, bbox4 = cvzone.putTextRect(img, mcq.choices[3], [300, 300], 2, 2, offset=30, border=5)

    else:
        score = 0
        for mcq in mcqList:
            if mcq.answer == mcq.userAns:
                score += 1
        score = round((score / qTotal) * 100, 2)
        img, _ = cvzone.putTextRect(img, "Quiz Completed", [220, 250], 2, 2, offset=30, border=5)
        img, _ = cvzone.putTextRect(img, f'Your Score: {score}%', [200, 400], 2, 2, offset=30, border=5)

    # Resize the image frame to increase window size
    resized_img = cv2.resize(img, (800, 600))

    # Display the resized image frame
    cv2.imshow("Image", resized_img)
    
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
